<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# 自动驾驶 - 数据采集

如果您已经浏览了避碰示例，您应该熟悉以下三个步骤

1. 数据收集
2. 培训
3. 部署

#在这个笔记本中，我们将做同样的事情!不过，除了分类之外，您还将学习另一种基本技术，即**regression**，我们将使用它使JetBot能够沿着一条路(实际上是任何路径或目标点)前进.

1. 将JetBot放置在路径的不同位置(从中心偏移，不同角度等).

>记住从避免碰撞得出的经验，数据变化是关键!

2. 显示来自机器人3的实时摄像头输入.
3. 使用gamepad控制器，在图像上放置一个“绿点”，它对应于我们希望机器人移动的目标方向。
4. 将这个绿色点的X, Y值和机器人摄像头的图像一起存储

然后，在训练笔记本中，我们将训练一个神经网络来预测我们的标签的X, Y值。在现场演示中，我们将使用预测的X、Y值来计算一个近似的转向值(它不是一个“确切”的角度，因为这需要图像校准，但它与角度大致成正比，因此我们的控制器将正常工作).

那么，如何确定本例的目标位置呢?以下是我们认为可能有所帮助的指南

1. 看看摄像机的实时视频
2. 想象机器人应该遵循的路径(试着接近它需要的距离，以避免跑离道路等).
3. 将目标放置在尽可能远的路径上，这样机器人就可以直接冲向目标，而不会“跑离”道路.

> 例如，如果我们在一条非常直的路上，我们可以把它放在地平线上.如果我们在急转弯，它可能需要放在离机器人更近的地方，这样它就不会跑出边界.

假设我们的深度学习模型如预期的那样工作，这些标记指南应该确保以下几点:

1. 机器人可以安全直接向目标移动(不出界等)
2. 目标将沿着我们想象的道路不断前进

我们得到的，是一根大路上的胡萝卜,胡萝卜沿着我们想要的轨迹移动,深度学习决定把胡萝卜放在哪里，而JetBot只是跟着它行走.

### 数据采集实例视频

执行下面单元格代码块，我们可以在单元格下方看到一条youtube的视频的源,前提是我们得学会科学上网才行,如果不会科学上网的话我们会把
视频存放到Jetbot配套资料附件目录中供大家参考

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

### 导入库模块

因此，让我们从导入所有“数据收集”所需的库开始。我们将主要使用OpenCV对带有标签的图像进行可视化和保存。uuid、datetime等库用于映像命名。

In [ ]:
# 用于显示和小部件的IPython库
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
# JetBot的摄像头和电机接口
from jetbot import Robot, Camera, bgr8_to_jpeg
# 用于图像注释的Python 基础包
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time
from servoserial import ServoSerial
import threading
# 杀掉线程
import inspect
import ctypes

# 实时显示摄像头的数据

首先，让我们初始化并显示我们的相机.

我们使用JetBot中的Camera类来启用CSI MIPI Camera。我们的神经网络以224x224像素的图像作为输入.我们将相机设置为这个大小，以最小化数据集的文件大小(我们已经测试过它适用于此任务).在某些场景中，最好以较大的图像大小收集数据，然后将其缩小到所需的大小.

In [ ]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

# 创建手柄控制器实例
这一步类似于“手柄遥控”任务。在这个示例中，我们将使用gamepad控制器来标记图像。
我们要做的第一件事是创建Controller小部件的一个实例，我们将使用该小部件用“x”和“y”值来标记图像，如介绍中所述。
Controller小部件接受一个索引参数，该参数指定控制器的数量。如果你有多个控制器，或者一些游戏手柄作为多个控制器出现，这是非常有用的。要确定正在使用的控制器的索引，

然后在创建手柄实例之前我们还是按照我们之前刚开始学习使用遥控手柄的步骤进行配置:
1. 访问http://html5gamepad.com。
2. 按下你正在使用的游戏手柄上的按钮
3. 记住响应按键的gamepad的索引

接下来，我们将使用该索引创建并显示控制器.

In [ ]:
controller = widgets.Controller(index=0)
robot = Robot()
display(controller)

### 连接Gamepad控制器到标签图像

现在，即使我们已经连接了gamepad，我们还没有将控制器连接到label images!
我们将使用dlink函数将其连接到左右垂直轴。dlink函数与link函数不同，它允许我们在源和目标之间附加一个转换。

In [ ]:
widgets.jsdlink((controller.axes[2], 'value'), (x_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (y_slider, 'value'))

# 收集数据

下面的单元格代码将显示实时图像提要，以及我们保存的图像数量。我们存储目标X, Y的值

1. 把绿点放在目标上
2. 按下 第13个按键 进行保存

然后我们想要的数据将会保存到 ``dataset_xy`` 文件夹 

保存的文件命名格式为:
``xy_<x value>_<y value>_<uuid>.jpg``

当我们训练时，我们加载图像并解析文件名中的x、y值

In [ ]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

# Jetbot Yahboom 手柄 L侧1键保存数据
controller.buttons[4].observe(save_snapshot, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

如果你觉得手动放置Jetbot采集数据麻烦的话，为了使用方便我们可以新开一条线程使用之前手柄的使用章节部分功能来通过手柄控制移动Jetbot采集数据

In [ ]:
def jetbot_motion():
    while 1:
        #小车左右DC motor
        if controller.axes[1].value <= 0.1:
            if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1 
                and controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1):
                robot.stop()
            else:
                robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
            
            time.sleep(0.01)
        else:
            robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
            time.sleep(0.01)
          #手柄操作代码---2(Xbox360手柄)
#         if controller.axes[1].value <= 0:
#             robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
#             time.sleep(0.01)
#         else:
#             robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
#             time.sleep(0.01)

开启线程

In [ ]:
thread1 = threading.Thread(target=jetbot_motion)
thread1.setDaemon(True)
thread1.start()

添加关闭线程的方法

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id") 
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        

def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

创建将Jetbot云台位置调节至自动驾驶角度的方法

In [ ]:
servo_device = ServoSerial() 
def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 2048
    servo_device.Servo_serial_control(1, 2048)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 1300)

调用上述方法将云台调节至自动驾驶角度

In [ ]:
camservoInitFunction()

In [ ]:
stop_thread(thread1)

一旦你收集了足够的数据，我们需要将这些数据复制到我们的GPU桌面或云机器上进行培训。首先，我们可以调用下面的terminal命令将dataset文件夹备份压缩到一个zip文件中。

> # 如果你是在JetBot上进行培训，你可以直接跳过这一步!

通过运行下面的数据将数据备份到压缩文件

下面zip命令中的-r标志表示递归，这样我们就包含了所有嵌套的文件，-q标志表示静默，这样zip命令就不会打印任何输出

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}

执行完上面的单元格代码后你应该在Jupyter文件浏览器中看到一个名为road_following_<Date&Time>.zip的文件。您应该通过右键单击并选择download，使用Jupyter文件浏览器下载zip文件。